In [1]:
# Convolutional Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# Install Tensorflow from the website: https://www.tensorflow.org/versions/r0.12/get_started/os_setup.html

# Installing Keras
# pip install --upgrade keras

# Resolve OMP issue
# Error #15: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Enable GPU Support ###
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 25,
                         validation_data = test_set,
                         nb_val_samples = 2000)
                         

Using TensorFlow backend.
/opt/anaconda3/envs/cwenv/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
/opt/anaconda3/envs/cwenv/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
/opt/anaconda3/envs/cwenv/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/opt/anaconda3/envs/cwenv/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


/opt/anaconda3/envs/cwenv/lib/python3.7/site-packages/ipykernel_launcher.py:74: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/opt/anaconda3/envs/cwenv/lib/python3.7/site-packages/ipykernel_launcher.py:74: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=250, epochs=25, validation_steps=2000)`


Epoch 1/25
250/250 [==============================] - 214s 858ms/step - loss: 0.6715 - accuracy: 0.5853 - val_loss: 0.6111 - val_accuracy: 0.6729
Epoch 2/25
250/250 [==============================] - 214s 858ms/step - loss: 0.5963 - accuracy: 0.6801 - val_loss: 0.5827 - val_accuracy: 0.6939
Epoch 3/25
250/250 [==============================] - 216s 864ms/step - loss: 0.5590 - accuracy: 0.7088 - val_loss: 0.4899 - val_accuracy: 0.7301
Epoch 4/25
250/250 [==============================] - 212s 847ms/step - loss: 0.5297 - accuracy: 0.7334 - val_loss: 0.6379 - val_accuracy: 0.7316
Epoch 5/25
250/250 [==============================] - 209s 835ms/step - loss: 0.4989 - accuracy: 0.7571 - val_loss: 0.7235 - val_accuracy: 0.7555
Epoch 6/25
250/250 [==============================] - 208s 830ms/step - loss: 0.4812 - accuracy: 0.7669 - val_loss: 0.3318 - val_accuracy: 0.7724
Epoch 7/25
250/250 [==============================] - 208s 834ms/step - loss: 0.4735 - accuracy: 0.7776 - val_loss: 0.4556 -